# Coffee Price Asymmetry Analysis

This notebook examines asymmetric price transmission ("rockets and feathers") in the U.S. coffee market,
analyzing how retail ground coffee prices respond to changes in wholesale coffee futures prices.

**Methodology:**
- ECM model
- Symmetric and asymmetric specifications
<!-- - Robustness checks: lag selection, outlier trimming, placebo leads -->

**Data:**
- Retail prices: Nielsen scanner data (weekly, 2013-2023)
- Upstream costs: Coffee futures (ICE Coffee C)

In [48]:
# =============================================================================
# Imports and Configuration
# =============================================================================

from __future__ import annotations

import os
from pathlib import Path
from typing import TYPE_CHECKING

import duckdb
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint

if TYPE_CHECKING:
    from statsmodels.regression.linear_model import RegressionResultsWrapper

# =============================================================================
# Configuration
# =============================================================================

# Berkeley brand colors
BERKELEY_BLUE = "#002676"
CALIFORNIA_GOLD = "#FDB515"
DARK_GREEN = "#00553A"
DARK_PURPLE = "#431170"
MEDIUM_ROSE = "#E7115E"

# Paths - adjust these for your environment
DATA_FOLDER = Path("/home/akimovh/rockets_feathers/coffee/data")
PLOT_FOLDER = Path("/home/akimovh/rockets_feathers/coffee/plots")

# Database connection
con = duckdb.connect(str(DATA_FOLDER / "coffee_nielsen" / "coffee_nielsen.duckdb"))
con.execute(f"PRAGMA threads={os.cpu_count()}")

## 1. Data Preparation

Merge weekly retail prices from Nielsen scanner data with coffee futures prices.

In [49]:
# =============================================================================
# Load and Process Futures (Wholesale Cost) Data
# =============================================================================

futures_path = DATA_FOLDER / "invest" / "coffee_futures.csv"
futures_df = pd.read_csv(futures_path, usecols=["Date", "Price"])
futures_df.columns = ["date", "up_price"]
futures_df["date"] = pd.to_datetime(futures_df["date"])
futures_df = futures_df.sort_values("date").set_index("date")

# Resample to weekly frequency (Saturday end) and convert to cents per ounce
# Coffee futures are quoted in cents per pound; divide by 16 for cents per ounce
cost_df = (
    futures_df["up_price"]
    .resample("W-SAT", label="right", closed="right")
    .mean()
    .reset_index()
    .rename(columns={"date": "date", "up_price": "cost"})
)
# cost_df = cost_df.query("date >= '2013-01-01' and date < '2024-01-01'").copy()
cost_df["cost"] = cost_df["cost"] / 16

# Apply 13-week centered moving average (approximately 1 quarter)
# cost_df["cost"] = cost_df["cost"].rolling(window=13, center=True).mean()

# Drop rows with NaN from the centered MA (first and last 6 observations)
cost_df = cost_df.dropna().copy()

# =============================================================================
# Load Retail Price Data from Nielsen
# =============================================================================

price_df = con.execute(
    """
    SELECT week_end AS date,
           avg_price * 100 AS price
    FROM GROUND_COFFEE_PRICE_AGG

    """
).df()

# =============================================================================
# Merge Datasets
# =============================================================================

df_weekly = pd.merge(price_df, cost_df, on="date", how="inner")

print(f"Sample period: {df_weekly['date'].min()} to {df_weekly['date'].max()}")
print(f"Observations: {len(df_weekly)}")
df_weekly['lp'] = np.log(df_weekly['price'])
df_weekly['lc'] = np.log(df_weekly['cost'])
df_weekly.head()

Sample period: 2013-01-05 00:00:00 to 2023-12-30 00:00:00
Observations: 574


,date,price,cost,lp,lc
0,2013-01-05,41.438943,9.172656,3.724221,2.216227
1,2013-01-12,39.702247,9.368125,3.681408,2.237313
2,2013-01-19,40.104088,9.632500,3.691478,2.265143
3,2013-01-26,39.584277,9.278906,3.678432,2.227744
4,2013-02-02,40.349617,9.267500,3.697582,2.226514


In [50]:
# =============================================================================
# Time Series Plot: Retail vs. Futures Prices (Side by Side)
# =============================================================================

from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Retail Price vs. Coffee Futures Price", 
                    "Log Retail Price vs. Log Coffee Futures Price"),
    horizontal_spacing=0.12
)

# Left plot: Original prices
fig.add_trace(
    go.Scatter(
        x=df_weekly["date"],
        y=df_weekly["price"],
        mode="lines",
        name="Retail",
        line=dict(color=BERKELEY_BLUE, width=3),
        showlegend=True,
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_weekly["date"],
        y=df_weekly["cost"],
        mode="lines",
        name="Futures",
        line=dict(color=CALIFORNIA_GOLD, width=3),
        showlegend=True,
    ),
    row=1, col=1
)

# Right plot: Log prices
fig.add_trace(
    go.Scatter(
        x=df_weekly["date"],
        y=df_weekly["lp"],
        mode="lines",
        name="Retail",
        line=dict(color=BERKELEY_BLUE, width=3),
        showlegend=False,
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=df_weekly["date"],
        y=df_weekly["lc"],
        mode="lines",
        name="Futures",
        line=dict(color=CALIFORNIA_GOLD, width=3),
        showlegend=False,
    ),
    row=1, col=2
)

fig.update_xaxes(
    title_text="Date",
    showgrid=True, gridcolor="lightgray", zeroline=False,
    linecolor="black", linewidth=1, mirror=True,
    ticks="outside", tickwidth=1, tickcolor="black",
    row=1, col=1
)

fig.update_xaxes(
    title_text="Date",
    showgrid=True, gridcolor="lightgray", zeroline=False,
    linecolor="black", linewidth=1, mirror=True,
    ticks="outside", tickwidth=1, tickcolor="black",
    row=1, col=2
)

fig.update_yaxes(
    title_text="Price (Cents per Ounce)",
    showgrid=True, gridcolor="lightgray", zeroline=False,
    linecolor="black", linewidth=1, mirror=True,
    ticks="outside", tickwidth=1, tickcolor="black",
    row=1, col=1
)

fig.update_yaxes(
    title_text="Log Price",
    showgrid=True, gridcolor="lightgray", zeroline=False,
    linecolor="black", linewidth=1, mirror=True,
    ticks="outside", tickwidth=1, tickcolor="black",
    row=1, col=2
)

fig.update_layout(
    font=dict(family="Times New Roman", size=14, color="black"),
    template="simple_white",
    legend=dict(
        x=1.02, y=1, xanchor="left", yanchor="top",
        font=dict(size=15), bgcolor="rgba(255,255,255,0)", bordercolor="rgba(0,0,0,0)",
    ),
    margin=dict(r=150),
    width=1400,
    height=520,
)

fig.show()
fig.write_image(PLOT_FOLDER / "Price_Cost.png")

In [51]:
# =============================================================================
# Time Series Plot: Retail vs. Futures Prices
# =============================================================================

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_weekly["date"],
        y=df_weekly["price"],
        mode="lines",
        name="Retail",
        line=dict(color=BERKELEY_BLUE, width=3),
    )
)


fig.update_layout(
    title="Retail Price for Ground Coffee vs. Coffee Futures Price",
    xaxis_title="Date",
    yaxis_title="Price (Cents per Ounce)",
    font=dict(family="Times New Roman", size=14, color="black"),
    template="simple_white",
    xaxis=dict(
        showgrid=True, gridcolor="lightgray", zeroline=False,
        linecolor="black", linewidth=1, mirror=True,
        ticks="outside", tickwidth=1, tickcolor="black",
    ),
    yaxis=dict(
        showgrid=True, gridcolor="lightgray", zeroline=False,
        linecolor="black", linewidth=1, mirror=True,
        ticks="outside", tickwidth=1, tickcolor="black",
    ),
    legend=dict(
        x=1.02, y=1, xanchor="left", yanchor="top",
        font=dict(size=15), bgcolor="rgba(255,255,255,0)", bordercolor="rgba(0,0,0,0)",
    ),
    margin=dict(r=150),
    width=850,
    height=520,
)

fig.show()
fig.write_image(PLOT_FOLDER / "Price_Cost.png")

In [52]:
def check_stationarity(series, name):
    result = adfuller(series.dropna())
    print(f"--- {name} ---")
    print(f"ADF Statistic: {result[0]:.4f}")
    print(f"P-Value:       {result[1]:.4f}")
    if result[1] < 0.05:
        print(">> STATIONARY (I(0))")
    else:
        print(">> NON-STATIONARY (Likely I(1))")
    print("\n")

check_stationarity(df_weekly['lp'], "Log price")
check_stationarity(df_weekly['lc'], "Log cost")

check_stationarity(df_weekly['lp'].diff(), "d_Log_price")
check_stationarity(df_weekly['lc'].diff(), "d_Log_cost")

--- Log price ---
ADF Statistic: 0.0404
P-Value:       0.9617
>> NON-STATIONARY (Likely I(1))


--- Log cost ---
ADF Statistic: -1.8456
P-Value:       0.3580
>> NON-STATIONARY (Likely I(1))


--- d_Log_price ---
ADF Statistic: -12.0307
P-Value:       0.0000
>> STATIONARY (I(0))


--- d_Log_cost ---
ADF Statistic: -16.7426
P-Value:       0.0000
>> STATIONARY (I(0))




In [53]:
from statsmodels.tsa.stattools import coint

for tr in ["c", "ct", "ctt"]:
    stat, pval, _ = coint(df_weekly["lp"], df_weekly["lc"], trend=tr)
    print(tr, pval)


c 0.7817419261484544
ct 0.2973191998711012
ctt 0.3387181263902371


# Borenstein-style ECM

In [42]:
def estimate_passthrough_ecm(
    df_weekly: pd.DataFrame,
    K_c: int = 20,
    K_p: int = 4,
    L_leads: int = 0,
    outlier_winsorize: bool = False,
    hac_lags: int = 8,
) -> tuple[RegressionResultsWrapper, list[str], list[str], str]:
    """
    Estimate ECM pass-through model with cointegrating relation.

    Parameters
    ----------
    df_weekly : pd.DataFrame
        Weekly data with 'price' and 'cost' columns.
    K_c : int
        Number of lags to include for Δlog(cost) (0 to K_c).
    K_p : int
        Number of lags to include for Δlog(price) (1 to K_p).
    L_leads : int
        Number of leads for placebo test.
    outlier_winsorize : bool
        If True, winsorize Δlog(cost) at 1st and 99th percentiles.
    hac_lags : int
        Bandwidth for HAC (Newey-West) standard errors.

    Returns
    -------
    model : RegressionResultsWrapper
        Fitted OLS model with HAC standard errors.
    lead_vars : list[str]
        Names of lead variables.
    lag_vars : list[str]
        Names of lag variables (both cost and price).
    ec_var : str
        Name of error correction term variable.
    """
    df = df_weekly.copy()
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date").reset_index(drop=True)

    # Log transformations and first differences
    df["lp"] = np.log(df["price"])
    df["lc"] = np.log(df["cost"])
    df["d_lp"] = df["lp"].diff()
    df["d_lc"] = df["lc"].diff()

    # Winsorize outliers if requested
    if outlier_winsorize:
        p01, p99 = df["d_lc"].quantile([0.01, 0.99])
        n_capped = ((df["d_lc"] < p01) | (df["d_lc"] > p99)).sum()
        df["d_lc"] = df["d_lc"].clip(lower=p01, upper=p99)
        print(f"Winsorization: capped {n_capped} observations at 1st/99th percentiles")

    # Error correction term: EC_{t-1} = log(p_{t-1}) - beta_0 - beta_c * log(c_{t-1})
    # First, estimate cointegrating relationship to get coefficients
    df_coint = df[["lp", "lc"]].dropna()
    X_coint = sm.add_constant(df_coint["lc"])
    y_coint = df_coint["lp"]
    coint_model = sm.OLS(y_coint, X_coint).fit()
    
    beta_0 = coint_model.params["const"]
    beta_c = coint_model.params["lc"]
    
    # Create error correction term (including intercept)
    df["ec_term"] = df["lp"] - beta_0 - beta_c * df["lc"]
    df["ec_term_lag1"] = df["ec_term"].shift(1)

    print(f"\nCointegrating relationship: log(p) = {beta_0:.4f} + {beta_c:.4f}*log(c)")
    
    # Test stationarity of error correction term using ADF
    from statsmodels.tsa.stattools import adfuller
    ec_clean = df["ec_term"].dropna()
    adf_result = adfuller(ec_clean, maxlag=12, regression='c')
    
    print(f"\nADF test for EC term stationarity:")
    print(f"  ADF statistic: {adf_result[0]:.4f}")
    print(f"  p-value: {adf_result[1]:.4f}")
    print(f"  Critical values: 1%={adf_result[4]['1%']:.4f}, "
          f"5%={adf_result[4]['5%']:.4f}, 10%={adf_result[4]['10%']:.4f}")
    if adf_result[1] < 0.05:
        print(f"  → EC term is stationary (reject unit root at 5% level)")
    else:
        print(f"  → EC term may have unit root (cannot reject at 5% level)")

    # Create lags for Δlog(cost)
    for lag in range(K_c + 1):
        df[f"d_lc_l{lag}"] = df["d_lc"].shift(lag)

    # Create lags for Δlog(price)
    for lag in range(1, K_p + 1):  # Start from lag 1 (lag 0 is the dependent variable)
        df[f"d_lp_l{lag}"] = df["d_lp"].shift(lag)

    # Create leads (placebo) for Δlog(cost)
    for lead in range(1, L_leads + 1):
        df[f"d_lc_lead{lead}"] = df["d_lc"].shift(-lead)

    # Month fixed effects
    df["month"] = df["date"].dt.month
    month_dummies = pd.get_dummies(df["month"], prefix="m", drop_first=True, dtype=int)
    df = pd.concat([df, month_dummies], axis=1)

    # Drop NAs and estimate
    df = df.dropna().copy()

    # Build regressor lists
    lag_vars_cost = [f"d_lc_l{lag}" for lag in range(K_c + 1)]
    lag_vars_price = [f"d_lp_l{lag}" for lag in range(1, K_p + 1)]
    lag_vars = lag_vars_cost + lag_vars_price
    lead_vars = [f"d_lc_lead{lead}" for lead in range(1, L_leads + 1)]
    ec_var = "ec_term_lag1"
    
    X_cols = lead_vars + lag_vars + [ec_var] + list(month_dummies.columns)

    y = df["d_lp"]
    X = sm.add_constant(df[X_cols])

    model = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": hac_lags})

    return model, lead_vars, lag_vars, ec_var

In [43]:
K_c = 20
K_p = 4
model, lead_vars, lag_vars, ec_var = estimate_passthrough_ecm(df_weekly)
print(model.summary())


Cointegrating relationship: log(p) = 3.8456 + -0.0853*log(c)

ADF test for EC term stationarity:
  ADF statistic: -3.9096
  p-value: 0.0020
  Critical values: 1%=-3.4463, 5%=-2.8686, 10%=-2.5705
  → EC term is stationary (reject unit root at 5% level)
                            OLS Regression Results                            
Dep. Variable:                   d_lp   R-squared:                       0.376
Model:                            OLS   Adj. R-squared:                  0.311
Method:                 Least Squares   F-statistic:                     15.08
Date:                Sun, 01 Feb 2026   Prob (F-statistic):           7.34e-53
Time:                        01:43:20   Log-Likelihood:                 933.23
No. Observations:                 396   AIC:                            -1790.
Df Residuals:                     358   BIC:                            -1639.
Df Model:                          37                                         
Covariance Type:                  HA

In [44]:
# Extract coefficients from the model
params = model.params

beta = {}  # Cost (d_lc) coefficients
gamma = {}  # Price (d_lp) coefficients

periods = 12  # up to a year

# Extract cost lag coefficients
for i in range(K_c + 1):
    beta[i] = params.get(f'd_lc_l{i}', 0)

# Extract price lag coefficients
for i in range(1, K_p + 1):
    gamma[i] = params.get(f'd_lp_l{i}', 0)

# Extract error correction coefficient
theta = params.get(ec_var, 0)

# Note: phi would be 0 in long-run equilibrium for our EC term specification
phi = 0

In [45]:
# Initialize cumulative pass-through
B = [beta[0]]

for k in range(1, periods):
    # Direct effect from cost change at lag k
    term_beta = beta.get(k, 0)
    
    # Error correction term
    term_ect = theta * (B[k-1] - phi)
    
    # Autoregressive feedback from previous price changes
    term_ar = 0
    for j in range(1, min(K_p + 1, k + 1)):
        if k - j >= 0:
            if k - j == 0:
                prev_change = B[0]
            else:
                prev_change = B[k-j] - B[k-j-1]
            term_ar += gamma.get(j, 0) * prev_change
    
    # Total change in pass-through at period k
    delta_B_k = term_beta + term_ect + term_ar
    B.append(B[k-1] + delta_B_k)

# Add initial zero point
B.insert(0, 0)

In [46]:
x = np.arange(-1, len(B)-1, 1)

# UC Berkeley colors
berkeley_blue = "#002676"

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=B,
    mode="lines",
    name="Cumulative Pass-Through",
    line=dict(color=berkeley_blue, width=3.5)
))

fig.update_layout(
    title="Cumulative Pass-Through Function",
    xaxis_title="Weeks After Cost Shock",
    yaxis_title="Pass-Through Rate",

    # ECONOMETRICA / QJE STYLE
    font=dict(
        family="Times New Roman",
        size=14,
        color="black"
    ),
    template="simple_white",

    # Axes styling
    xaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False,
        linecolor="black",
        linewidth=1,
        mirror=True,
        ticks="outside",
        tickwidth=1,
        tickcolor="black"
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False,
        linecolor="black",
        linewidth=1,
        mirror=True,
        ticks="outside",
        tickwidth=1,
        tickcolor="black"
    ),

    # Legend OUTSIDE (right)
    legend=dict(
        x=1.02,
        y=1,
        xanchor="left",
        yanchor="top",
        font=dict(size=15),
        bgcolor="rgba(255,255,255,0)",
        bordercolor="rgba(0,0,0,0)"
    ),

    margin=dict(r=150),
    width=850,
    height=520
)

fig.show()

# Uncomment to save
# fig.write_image("path/to/plots/cumulative_passthrough.png")

## 3. Symmetric Pass-Through Model

Estimate the distributed lag model:

$$\Delta \ln(p_t) = \alpha + \sum_{\ell=0}^{K} \beta_\ell \Delta \ln(c_{t-\ell}) + \gamma_m + \varepsilon_t$$

where $p_t$ is retail price, $c_t$ is wholesale cost, and $\gamma_m$ are month fixed effects.

## 5. Asymmetric Pass-Through Model

Test for "rockets and feathers" by estimating separate coefficients for cost increases and decreases:

$$\Delta \ln(p_t) = \alpha + \sum_{\ell=0}^{K} \beta^+_\ell (\Delta \ln c_{t-\ell})^+ + \sum_{\ell=0}^{K} \beta^-_\ell (\Delta \ln c_{t-\ell})^- + \gamma_m + \varepsilon_t$$

In [ ]:
def estimate_passthrough_asymmetric(
    df_weekly: pd.DataFrame,
    K_lags: int = 20,
    L_leads: int = 0,
    outlier_winsorize: bool = False,
    hac_lags: int = 8,
) -> tuple[RegressionResultsWrapper, list[str], list[str], list[str], list[str]]:
    """
    Estimate distributed-lag model with ASYMMETRIC pass-through.

    Returns
    -------
    model : RegressionResultsWrapper
        Fitted OLS model.
    lead_vars_pos, lead_vars_neg : list[str]
        Names of positive/negative lead variables.
    lag_vars_pos, lag_vars_neg : list[str]
        Names of positive/negative lag variables.
    """
    df = df_weekly.copy()
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date")

    # Log transformations and first differences
    df["lp"] = np.log(df["price"])
    df["lc"] = np.log(df["cost"])
    df["d_lp"] = df["lp"].diff()
    df["d_lc"] = df["lc"].diff()

    if outlier_winsorize:
        p01, p99 = df["d_lc"].quantile([0.01, 0.99])
        df["d_lc"] = df["d_lc"].clip(lower=p01, upper=p99)

    # Split into positive and negative changes
    df["d_lc_pos"] = np.maximum(0, df["d_lc"])
    df["d_lc_neg"] = np.minimum(0, df["d_lc"])

    # Create lags
    for lag in range(K_lags + 1):
        df[f"d_lc_pos_l{lag}"] = df["d_lc_pos"].shift(lag)
        df[f"d_lc_neg_l{lag}"] = df["d_lc_neg"].shift(lag)

    # Create leads (placebo)
    for lead in range(1, L_leads + 1):
        df[f"d_lc_pos_lead{lead}"] = df["d_lc_pos"].shift(-lead)
        df[f"d_lc_neg_lead{lead}"] = df["d_lc_neg"].shift(-lead)

    # Month fixed effects
    df["month"] = df["date"].dt.month
    month_dummies = pd.get_dummies(df["month"], prefix="m", drop_first=True, dtype=int)
    df = pd.concat([df, month_dummies], axis=1)

    df = df.dropna().copy()

    lag_vars_pos = [f"d_lc_pos_l{lag}" for lag in range(K_lags + 1)]
    lag_vars_neg = [f"d_lc_neg_l{lag}" for lag in range(K_lags + 1)]
    lead_vars_pos = [f"d_lc_pos_lead{lead}" for lead in range(1, L_leads + 1)]
    lead_vars_neg = [f"d_lc_neg_lead{lead}" for lead in range(1, L_leads + 1)]

    X_cols = lead_vars_pos + lead_vars_neg + lag_vars_pos + lag_vars_neg + list(month_dummies.columns)
    y = df["d_lp"]
    X = sm.add_constant(df[X_cols])

    model = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": hac_lags})

    return model, lead_vars_pos, lead_vars_neg, lag_vars_pos, lag_vars_neg

In [ ]:
# =============================================================================
# Asymmetric Model Estimation
# =============================================================================

K = 20
m_asym, lead_pos, lead_neg, lag_pos, lag_neg = estimate_passthrough_asymmetric(
    df_weekly, K_lags=K, L_leads=0
)

In [ ]:
# =============================================================================
# Joint Asymmetry Test: H₀: β⁺_ℓ = β⁻_ℓ for all ℓ
# =============================================================================

restrictions = [f"{lag_pos[l]} - {lag_neg[l]} = 0" for l in range(K + 1)]
hypotheses = ", ".join(restrictions)
f_test_asym = m_asym.f_test(hypotheses)

print("\n" + "=" * 80)
print("JOINT ASYMMETRY TEST")
print("=" * 80)
print(f"H₀: β⁺_ℓ = β⁻_ℓ for all ℓ = 0,...,{K}")
print(f"F-statistic: {float(f_test_asym.fvalue):.4f}")
print(f"p-value:     {f_test_asym.pvalue:.6f}")
print(f"df:          ({f_test_asym.df_num}, {f_test_asym.df_denom})")

if f_test_asym.pvalue < 0.05:
    print("\n✓ ASYMMETRY DETECTED (p < 0.05): Pass-through differs for increases vs. decreases")
else:
    print("\n✗ No significant asymmetry detected (p ≥ 0.05)")


JOINT ASYMMETRY TEST
H₀: β⁺_ℓ = β⁻_ℓ for all ℓ = 0,...,20
F-statistic: 2.0614
p-value:     0.003861
df:          (21.0, 499.0)

✓ ASYMMETRY DETECTED (p < 0.05): Pass-through differs for increases vs. decreases


In [ ]:
# =============================================================================
# Asymmetric Cumulative Pass-Through Curves
# =============================================================================

beta_pos = m_asym.params[lag_pos].values
beta_neg = m_asym.params[lag_neg].values

cumulative_pt_pos = np.cumsum(beta_pos)
cumulative_pt_neg = np.cumsum(beta_neg)

# Standard errors for cumulative effects
vcov = m_asym.cov_params()

vcov_pos = vcov.loc[lag_pos, lag_pos]
cumulative_se_pos = np.array([
    np.sqrt(np.ones(h + 1) @ vcov_pos.iloc[: h + 1, : h + 1] @ np.ones(h + 1))
    for h in range(K + 1)
])

vcov_neg = vcov.loc[lag_neg, lag_neg]
cumulative_se_neg = np.array([
    np.sqrt(np.ones(h + 1) @ vcov_neg.iloc[: h + 1, : h + 1] @ np.ones(h + 1))
    for h in range(K + 1)
])

# 95% CIs
ci_pos_lower = cumulative_pt_pos - 1.96 * cumulative_se_pos
ci_pos_upper = cumulative_pt_pos + 1.96 * cumulative_se_pos
ci_neg_lower = cumulative_pt_neg - 1.96 * cumulative_se_neg
ci_neg_upper = cumulative_pt_neg + 1.96 * cumulative_se_neg

# Plot
horizons = np.arange(K + 1)

fig = go.Figure()

# CI for cost increases
fig.add_trace(
    go.Scatter(
        x=np.concatenate([horizons, horizons[::-1]]),
        y=np.concatenate([ci_pos_upper, ci_pos_lower[::-1]]),
        fill="toself",
        fillcolor="rgba(231, 17, 94, 0.2)",
        line=dict(color="rgba(255,255,255,0)"),
        showlegend=True,
        name="95% CI (Increases)",
        hoverinfo="skip",
    )
)

# CI for cost decreases
fig.add_trace(
    go.Scatter(
        x=np.concatenate([horizons, horizons[::-1]]),
        y=np.concatenate([ci_neg_upper, ci_neg_lower[::-1]]),
        fill="toself",
        fillcolor="rgba(0, 38, 118, 0.2)",
        line=dict(color="rgba(255,255,255,0)"),
        showlegend=True,
        name="95% CI (Decreases)",
        hoverinfo="skip",
    )
)

# Point estimates
fig.add_trace(
    go.Scatter(
        x=horizons,
        y=cumulative_pt_pos,
        mode="lines+markers",
        name="Cost Increases",
        line=dict(color=MEDIUM_ROSE, width=3),
        marker=dict(size=6, color=MEDIUM_ROSE),
    )
)

fig.add_trace(
    go.Scatter(
        x=horizons,
        y=cumulative_pt_neg,
        mode="lines+markers",
        name="Cost Decreases",
        line=dict(color=BERKELEY_BLUE, width=3),
        marker=dict(size=6, color=BERKELEY_BLUE),
    )
)

fig.add_hline(y=0, line_dash="dash", line_color="gray", line_width=1.5)

fig.update_layout(
    title="Asymmetric Cumulative Pass-Through: Cost Increases vs. Decreases",
    xaxis_title="Weeks after cost shock (h)",
    yaxis_title="Cumulative pass-through",
    font=dict(family="Times New Roman", size=14, color="black"),
    template="simple_white",
    xaxis=dict(
        showgrid=True, gridcolor="lightgray", zeroline=False,
        linecolor="black", linewidth=1, mirror=True,
        ticks="outside", tickwidth=1, tickcolor="black",
    ),
    yaxis=dict(
        showgrid=True, gridcolor="lightgray", zeroline=False,
        linecolor="black", linewidth=1, mirror=True,
        ticks="outside", tickwidth=1, tickcolor="black",
    ),
    legend=dict(
        x=1.02, y=1, xanchor="left", yanchor="top",
        font=dict(size=15), bgcolor="rgba(255,255,255,0)", bordercolor="rgba(0,0,0,0)",
    ),
    margin=dict(r=200),
    width=850,
    height=520,
)

fig.show()
fig.write_image(PLOT_FOLDER / f"Asymmetric_PassThrough_{K}.png")

In [38]:
con.close()